In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import random
import pandas as pd
from tqdm import tqdm
import numpy as np
import cv2
import torch
import re
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!unzip "/content/drive/MyDrive/archive.zip"

In [ ]:
categories = []
cata_dict = {}
dict_cata = {}
c = 0
with open('/content/tiny-imagenet-200/wnids.txt', encoding = 'utf8') as f:
    for line in tqdm(f):
        categories.append(line[:-1])
        cata_dict[line[:-1]] = c
        dict_cata[c] = line[:-1]
        c=c+1

In [ ]:
path = "/content/tiny-imagenet-200/train"
train_pairs = []
for category in tqdm(os.listdir(path)):
    '''


                    CODE IN COMMENTS WILL BLUR OUT THE BACKGROUND OF THE TRAINING SAMPLES USING THE 
                    BOUNDED BOX INFORMATION PROVIDED IN EVERY FOLDER, THE IDEA BEHIND THIS BEING THAT
                    THE MODEL SHOULD PICK UP PATTERNS ONLY FROM THE REGION OF INTEREST AND NOT THE BACKGROUND

                    DOING THIS RESULTS IN FASTER DECREASE IN TRAINING LOSS, BUT THE DIFFRENCE IN VALIDATION LOSS 
                    AND TRAINING LOSS IS HIGH, AS COMPARED TO USE OF NORMAL IMAGES.



    bbox = []
    bbox_path = os.path.join(path, category, category+"_boxes.txt")
    with open(bbox_path, encoding = 'utf8') as f:
        for line in f:
            coords = re.findall(r'\t(\d\d|\d)',line)
            bbox.append([int(coords[0]), int(coords[1]), int(coords[2]), int(coords[3])])
    '''
    img_path = os.path.join(path, category, "images")
    c = 0
    for img in os.listdir(img_path):
        imgg = cv2.imread(os.path.join(img_path, img))
        '''
        blurred_img = cv2.blur(cv2.imread(os.path.join(img_path, img)), (8, 8))
        mask = np.zeros((64, 64, 3), dtype=np.uint8)
        r = bbox[c]
        x1 = int(r[0])
        y1 = int(r[1])
        x2 = int(r[2])
        y2 = int(r[3])
        mask = cv2.rectangle(cv2.imread(os.path.join(img_path, img)), (x1, y1), (x2, y2), (255, 255, 255), -1)
        out = np.where(mask==np.array([255, 255, 255]), imgg, blurred_img)
        c = c + 1
        '''
        x = (torch.from_numpy(np.array(imgg))).to(device)
        y = (torch.tensor(cata_dict.get(img[:9])).reshape(-1)).to(device)
        train_pairs.append([x, y])




In [5]:
val_path = "/content/tiny-imagenet-200/val"
images = os.listdir(os.path.join(val_path,"images"))
val_pairs = []
val_labels = []
with open(os.path.join(val_path, "val_annotations.txt"), encoding = 'utf8') as f:
        for line in f:
            val_labels.append(re.findall(r'n\d\d\d\d\d\d\d\d', line)[0])
c = 0
for img in images:
    imgg = cv2.imread(os.path.join(val_path, "images", img))
    x = (torch.from_numpy(np.array(imgg))).to(device)
    y = (torch.tensor(cata_dict.get(val_labels[c])).reshape(-1)).to(device)
    val_pairs.append([x, y])
    c = c+1


In [6]:
class AlexNet(nn.Module):

    def __init__(self, num_classes=200):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [7]:
def train(input_tensor_batch, target_tensor_batch, AlexNet, optimizer, criterion, batch_size):
    
    optimizer.zero_grad()

    loss = 0

    outputs = AlexNet(input_tensor_batch.permute(0,3,1,2).float())
    
    loss = criterion(outputs, target_tensor_batch.reshape(-1))

    loss.backward()

    optimizer.step()

    return loss.item()

In [ ]:
cata_names = {}
with open('/content/tiny-imagenet-200/words.txt', encoding = 'utf8') as f:
    for line in tqdm(f):
        if cata_dict.get(line[:9])!=None:
            cata_names[line[:9]] = line[10:-1]

In [19]:
def trainIters(AlexNet, epochs, batch_size, print_every):
    
    optimizer = optim.Adam(AlexNet.parameters(), lr=0.0001)
    criterion = nn.CrossEntropyLoss()

    for epo in tqdm(range(epochs)):
        random.shuffle(train_pairs)
        startt = 0
        endd = batch_size
        for it in range(int(100000/batch_size)):
            
            input_tensor = torch.zeros((batch_size,64,64,3), device=device, dtype=torch.long)
            target_tensor = torch.zeros((batch_size,1), device=device, dtype= torch.long)
            training_batch = train_pairs[startt:endd]
            
            for batch in range(batch_size):

                input_tensor[batch] = training_batch[batch][0]
                target_tensor[batch] = training_batch[batch][1]
                
            loss = train(input_tensor, target_tensor, AlexNet, optimizer, criterion, batch_size)
            
            startt = endd
            endd += batch_size
            
            if it%print_every==0:
                print("====== Batch Loss is ====== "+str(loss))
            print_loss_total = 0
        
            
        print('')
        print('')
        
        random.shuffle(val_pairs)
    
        input_tensor = torch.zeros((batch_size,64,64,3), device=device, dtype=torch.long)
        target_tensor = torch.zeros((batch_size,1), device=device, dtype= torch.long)
        val_batch = val_pairs[:batch_size]
            
        for batch in range(batch_size):

            input_tensor[batch] = val_batch[batch][0]
            target_tensor[batch] = val_batch[batch][1]
                
        loss = train(input_tensor, target_tensor, AlexNet, optimizer, criterion, batch_size)
        print("====== Val Loss is ====== "+str(loss))
        print('')
        print('')
        print("======================== EPOCH FINISHED =========================")
        print('')
        print('')

    right = 0
    for val in val_pairs:
        input_tensor = torch.zeros((1,64,64,3), device=device, dtype=torch.long)
        input_tensor[0] = val_pairs[0]
        input_tensor = input_tensor.permute(0,3,1,2).float()
        output = AlexNet(input_tensor)
        _, ind = torch.max(output.reshape(-1),0)
        if ind==(int(val_pairs[val][1])):
            right = right + 1

    print("Validation accuracy is : "+str(right/100))

    print("==============TEST SET EXAMPLES==============")
    print("")

    test_path = '/content/tiny-imagenet-200/test/images'
    eg = 0
    for i in os.listdir(test_path):
        img = cv2.imread(os.path.join(test_path, i))
        test_tensor = torch.zeros((1,64,64,3), device=device, dtype=torch.long)
        test_tensor[0] = (torch.from_numpy(np.array(img))).to(device)
        test_tensor = test_tensor.permute(0,3,1,2)
        output = AlexNet(test_tensor.float())
        _, ind = torch.max(output.reshape(-1),0)
        print(cata_names.get(dict_cata.get(int(ind))))
        plt.imshow(img)
        plt.show()
        eg = eg + 1
        if eg>=50:
          break



In [ ]:
classes = 200
epochs = 50
batch_size = 40
print_every = 500

AN = AlexNet(classes)
trainIters(AN, epochs, batch_size, print_every)